In [ ]:
# !pip install ultralytics ensemble-boxes

In [ ]:
from ultralytics import YOLO
from ensemble_boxes import weighted_boxes_fusion
import cv2
import matplotlib.pyplot as plt
import os
from glob import glob

In [ ]:
weights_path = '가중치들이 저장된 폴더 경로'
models = [YOLO(pt) for pt in glob(os.path.join(weights_path,'*'))]

In [ ]:
# YOLO output을 WBF에 맞게 변환 하는 함수
def boxes_and_scores(results):
    boxes = []
    scores = []
    labels = []
    for r in results[0].boxes.data.tolist():
        x1, y1, x2, y2, confidence, class_id = r
        boxes.append([x1 / image.shape[1], y1 / image.shape[0], x2 / image.shape[1], y2 / image.shape[0]])
        scores.append(confidence)
        labels.append(int(class_id))
    return boxes, scores, labels


In [ ]:
root = '추론 데이터 경로'
ufo = {'images':{}}

for img in os.listdir(root) :
    word = {}
    image_path = os.path.join(root,img)
    image = cv2.imread(image_path)

    # YOLO 모델로 예측 수행
    results = [model(image) for model in models]

    boxes_list, scores_list, labels_list = [], [], []

    for result in results :
        boxes, scores, labels = boxes_and_scores(result)
        boxes_list.append(boxes)
        scores_list.append(scores)
        labels_list.append(labels)

    iou_thr = 0.5  # IoU threshold
    skip_box_thr = 0.5  # Confidence threshold
    weights = [1, 1, 1]  # 모델들의 가중치

    fused_boxes, fused_scores, fused_labels = weighted_boxes_fusion(
        boxes_list, scores_list, labels_list, iou_thr=iou_thr, skip_box_thr=skip_box_thr, weights=weights
    )

    fused_boxes_abs = [
        [box[0] * image.shape[1], box[1] * image.shape[0], box[2] * image.shape[1], box[3] * image.shape[0]]
        for box in fused_boxes
    ]

    for idx,xyxy in enumerate(fused_boxes_abs) :
        x_min, y_min, x_max, y_max = xyxy
        rect = [[x_min, y_min], [x_max, y_min], [x_max, y_max], [x_min, y_max]]
        word[str(idx)] = {'points':rect}

    ufo['images'][img] = {}
    ufo['images'][img]['words'] = word

In [ ]:
import json
with open("yolo_output.json", "w") as json_file:
    json.dump(ufo, json_file,indent=4)